# Header

In [1]:
from bs4 import BeautifulSoup #html reader
import re
from bioservices import UniProt
import pandas as pd

In [7]:
#Example for sending query to Uniprot
# Make a link to the UniProt webservice
service = UniProt()

# Build a query string
#query = "Q9AJE3"
query = "gene:PDS"
# Send the query to UniProt, and catch the search result in a variable
result = service.search(query)

# Inspect the result
#print(result)
print(result.split('\n')[1])

A2XDA1	PDS_ORYSI	reviewed	15-cis-phytoene desaturase, chloroplastic/chromoplastic (EC 1.3.5.5) (Phytoene dehydrogenase) (Phytoene desaturase)	PDS1 PDS OsI_010044	Oryza sativa subsp. indica (Rice)	578


In [59]:
def read_file(files):
    '''Returns dict of all words in filehandle'''
    res={}
    for fh in files:
        fh = open(fh,'r')
        for line in fh:
            if not line:
                continue
            line=line.strip()
            if line not in res:
                res[line]=1
    return res

In [60]:
#Dict of english words
words = read_file(['words_alpha.txt'])
# Reviewed gene names
genes = read_file(['genename.txt','gene_oc.txt'])
print(len(words))
print(len(genes))

370103
1947214


In [21]:
#List of xml files
from os import walk
mypath = 'jki-sys-review-teixml'
files = []
for (_, _, filenames) in walk(mypath):
    files.extend(filenames)
    break

In [22]:
# Read csv-Result file
test_set = open('anhang4.csv','r')
test_set = test_set.readlines()
test={}
for line in test_set:
    line = line.split('\t')
    title = ' '.join(line[0].split(' ')[:5])
    if title not in test:
        test[title] = [line[1].strip()]
    else:
        test[title].append(line[1].strip())

In [85]:
# Predict gene names with regex
exclude = ['RNA', 'DNA', 'CRISPR']
find = pd.DataFrame(columns=['title','pred_pot_gn', 'pred_genenames', 'genenames'])
count=0
for key,value in test.items():
    key = key.replace('/',' ').replace('(','') #Different titles in csv and xml: CRISPR/Cas vs. CRISPR-Cas
    for file in files: # Looks for xml file with this key title in folder
        if key in file:
            fi = file            
            html_doc = open(f'jki-sys-review-teixml/{fi}','r')
            soup = BeautifulSoup(html_doc, 'html.parser')
            text = str(soup.abstract.get_text()) #Extracts Abstract paragraph, if exists
            if text=="":
                text = str(soup.div.get_text()) #Else extracts first paragraph
            text = re.split('[;,.\%/()\-′ ]',text) #Split special characters
            w_set=set(); pot=[] #Set of words that are not natural language
            for t in text:
                if t not in words and t!='':
                    w_set.add(t.strip('\n'))
            #print(w_set)
            for w in w_set:
                if len(w)<2: #Single character will not be considered
                    continue
                m = re.findall('[A-Z′]',w) #Finds words with more than 2 capital characters
                n = re.search('[0-9]',w) #Finds words that contains digits
                if len(m)>1 or (n and not w.isdigit()):
                    pot.append(w)
            gn=set()
            for p in pot:
                if p in exclude: continue
                if p in genes: gn.add(p)  
                elif re.search(r'^[A-Z][a-z]',p):
                    if p[2:] in genes or re.sub(r'[0-9]+$','',p[2:]) in genes:
                        gn.add(p)
                
                #    if nw in genes or re.sub(r'[0-9]+$','',nw) in genes:
                #            gn.add(p+f'({nw})')
                #else:
                #    w = [w[i:len(w)] for i in range(len(w))]
                #    for i in w:
                #        if i.lower() in genes: gn.append(i)
            
           
            find.loc[count] = [key, ' '.join(pot), ' '.join(gn),' '.join(value)]
            count+=1
            break
find

,title,pred_pot_gn,pred_genenames,genenames
0,Manipulating the Biosynthesis of Bioactive,sgRNA TRV TALEN ZFNs BIAs DNA DSBs OMT2 TALENs...,II SpCas9 Cas9 OMT2 TALENs,4′OMT2
1,Activity and specificity of TRV-mediated,cRNA pRNA2:pEBV::gRNA 2B CRISPRs tracrRNA RNA1...,1B II HDR RNA2 PDS PAM 1D RNA1 Cas9 1A TALENs ...,PDS3
2,DNA-free two-gene knockout in Chlamydomonas,RGENs CO −9 DiscussionTargeted DNA ZFNs RNAs R...,II ZEP CO RNP Cas9 TALENs CpFTSY,CpFTSY ZEP
3,CRISPR Cas9 activity in the rice,NGG sgRNAs RNAs Cas9 PAM *10 CRISPR,Cas9 PAM,BEIIb
4,CRISPR Cas9-mediated targeted mutagenesis of G...,sgRNA E116°20 DNA N39°58 T2 CRISPR GmFT2a T1 Cas9,Cas9 T1 T2,FT2a
5,Development of broad virus resistance,sgRNA eIF4E SNPs eif4e 4E CRISPR RNA T1 Cas9 T3,Cas9 T1 eif4e T3,elF4E
6,Targeted mutagenesis in cotton Gossypium,GhCLA1 RNAs PCR CRISPR Cas9 GhVP,Cas9 PCR GhVP GhCLA1,CLA1 VP
7,Efficient targeted mutagenesis in soybean,16g PDS TALENs GmPDS11 CRISPR GT AtU6 GmU6 Cas...,GmPDS11 PDS AtU6 GmU6 Cas9 TALENs GT GmPDS18,PDS11
8,Efficient targeted mutagenesis of rice,SSNs AT TTN SSN DNA ZFNs SaCas9 TALENs SpCas9 ...,II AT SpCas9 TTN SaCas9 FnCpf1 Cas9 PAM TALENs,STF1 NCED1 AO1
9,Multigene knockout utilizing off-target mutations,sgRNA DNA CDKB1 B2 CDKA2 CDKA1 CRISPR RNA CDK ...,CDK Cas9 B2,CDKB2


In [90]:
find.to_csv('result.csv',index=False, sep='\t')

1

In [79]:
re.sub(r'[0-9]+$','','SaCas99')
re.split('[a-z]','SaCas99')
'GmADF'[2:]

'ADF'